In [1]:
import numpy as np
import numba

import scipy.stats as st

import bokeh.plotting as bp
import bokeh.layouts as bl
import bokeh.io as bi
bi.output_notebook()

Loading BokehJS ...

Test data is Gaussian distributed with mean 10 and standard deviation 3.

In [2]:
data = np.random.normal(10, 3, size=50)

We'll fit a Gaussian distribution with uniform priors on µ and σ.

In [3]:
def logprior(x):
    if x[1] <= 0:
        return -np.inf
    return 0

def loglike(x, data):
    mu, sig = x
    return -np.sum((data - mu)**2) / 2 / sig**2 - len(data) * np.log(sig)

def logpost(x, data):
    lp = logprior(x)
    if lp == -np.inf:
        return -np.inf
    return lp + loglike(x, data)

To be maximally flexible, we would have the functions as inputs to the samplers, but this precludes use of Numba for JITting the samplers. As we will see, it is worth this inconvenience to get the speed.

In [27]:
def metropolis_sample(x0, sigma0, n_burn, n_samples, args=()):
    """
    Perform Metropolis sampling.
    """
    # Initialize
    n_accept = 0
    n_steps = 0
    x = np.copy(x0)
    sigma = sigma0
    logpost_current = logpost(x, *args)
    
    # Burn in
    while n_steps < n_burn:
        x, logpost_current, acccept = metropolis_step(
                                        x, logpost_current, sigma, args)
        n_steps += 1

    # Samples
    x_samples = np.empty((n_samples, len(x)))
    n_steps = 0
    while n_steps < n_samples:
        x, logpost_current, accept = metropolis_step(
                                        x, logpost_current, sigma, args)
        x_samples[n_steps,:] = x
        n_steps += 1
        n_accept += accept
        
    return x_samples, n_accept / n_samples


def metropolis_step(x, logpost_current, sigma, args=()):
    """
    Perform Metropolis step on continuous variables with Normal
    proposal distribution with sigma being the sqrt of the diagonal of the 
    covariance matrix. x is the current position of the walker.
    
    We do not use np.random.multivariate_normal() to generate our move
    because this is not supported by Numba.
    
    Because of the symmetry of the normal proposal distribution, the
    proposal probabilities cancel in the Metropolis ratio.
    """
    # Draw the next step
    x_next = np.array([np.random.normal(x[i], sigma[i]) for i in range(len(x))])
    
    # Compute log posterior
    logpost_new = logpost(x_next, *args)
    
    # Compute the log Metropolis ratio
    log_r = logpost_new - logpost_current

    # Accept or reject step
    if log_r >= 0 or np.random.random() < np.exp(log_r):
        return x_next, logpost_new, True
    else:
        return x, logpost_current, False

Ok, let's sample.

In [28]:
x0 = np.array([10, 3], dtype=np.float)
sigma0 = np.array([1, 1], dtype=np.float)
n_burn = 10000
n_samples = 10000
samples, accept_frac = metropolis_sample(x0, sigma0, n_burn, n_samples, (data,))

Let's look at the results. First, the acceptance fraction.

In [29]:
accept_frac

0.179

Let's look at traces.

In [30]:
p1 = bp.figure(height=300, width=600, y_axis_label='µ')
p1.line(x=np.arange(n_samples), y=samples[:,0])
p2 = bp.figure(height=300, width=600, y_axis_label='σ', x_axis_label='sample #')
p2.line(x=np.arange(n_samples), y=samples[:,1])
bi.show(bl.column(p1, p2))

And now let's look at how the samples compare to the theoretical distribution of the mean.

In [31]:
def ecdf(data):
    return np.sort(data), np.arange(1, len(data)+1) / len(data)

x, y = ecdf(samples[:,0])
y_theor = st.norm.cdf(x, data.mean(), data.std(ddof=1) / np.sqrt(len(data)))
p = bp.figure(height=300, width=300)
p.circle(x, y)
p.line(x, y_theor, color='tomato', line_width=3)
bi.show(p)

Very good.

Let's time it to see how long it took to get the samples.

In [32]:
%timeit metropolis_sample(x0, sigma0, n_burn, n_samples, (data,))

10 loops, best of 3: 148 ms per loop


Now let's do it with Numba. First, we'll JIT the log posterior.

In [33]:
@numba.jit(nopython=True)
def logprior(x):
    if x[1] <= 0:
        return -np.inf
    return 0

@numba.jit(nopython=True)
def loglike(x, data):
    mu, sig = x
    return -np.sum((data - mu)**2) / 2 / sig**2 - len(data) * np.log(sig)

@numba.jit(nopython=True)
def logpost(x, data):
    lp = logprior(x)
    if lp == -np.inf:
        return -np.inf
    return lp + loglike(x, data)

%timeit metropolis_sample(x0, sigma0, n_burn, n_samples, (data,))

1 loop, best of 3: 137 ms per loop


Not much. What if we Numba the whole thing? We have to mess a little bit the the `metropolis_step()` function because Numba doesn't like the list comprehension.

In [39]:
@numba.jit(nopython=True)
def metropolis_sample(x0, sigma0, n_burn, n_samples, args=()):
    """
    Perform Metropolis sampling.
    """
    # Initialize
    n_accept = 0
    n_steps = 0
    x = np.copy(x0)
    sigma = sigma0
    logpost_current = logpost(x, *args)
    
    # Burn in
    while n_steps < n_burn:
        x, logpost_current, acccept = metropolis_step(
                                        x, logpost_current, sigma, args)
        n_steps += 1

    # Samples
    x_samples = np.empty((n_samples, len(x)))
    n_steps = 0
    while n_steps < n_samples:
        x, logpost_current, accept = metropolis_step(
                                        x, logpost_current, sigma, args)
        x_samples[n_steps,:] = x
        n_steps += 1
        n_accept += accept
        
    return x_samples, n_accept / n_samples


@numba.jit(nopython=True)
def metropolis_step(x, logpost_current, sigma, args=()):
    """
    Perform Metropolis step on continuous variables with Normal
    proposal distribution with sigma being the sqrt of the diagonal of the 
    covariance matrix. x is the current position of the walker.
    
    We do not use np.random.multivariate_normal() to generate our move
    because this is not supported by Numba.
    
    Because of the symmetry of the normal proposal distribution, the
    proposal probabilities cancel in the Metropolis ratio.
    """
    # Draw the next step
    x_next = np.empty(len(x), dtype=x.dtype)
    for i in range(len(x)):
        x_next[i] = np.random.normal(x[i], sigma[i])
    
    # Compute log posterior
    logpost_new = logpost(x_next, *args)
    
    # Compute the log Metropolis ratio
    log_r = logpost_new - logpost_current

    # Accept or reject step
    if log_r >= 0 or np.random.random() < np.exp(log_r):
        return x_next, logpost_new, True
    else:
        return x, logpost_current, False
    
%timeit metropolis_sample(x0, sigma0, n_burn, n_samples, (data,))

The slowest run took 29.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 10.7 ms per loop


A factor of 15. This makes it worth Numba-ing.

There are two things left to do.
* Do tuning by messing with the variance in the Normal proposal distribution. We can use PyMC3's strategy: 

|Acceptance rate|Variance adaptation|
|:---:|:-------------------:|
| < 0.001        |x 0.1|
|< 0.05         |x 0.5|
|< 0.2          |x 0.9|
|> 0.5          |x 1.1|
|> 0.75         |x 2|
|> 0.95         |x 10|

* Write a sampler that allows for discrete variables. This is actually useful since the affine invariant sampler of `emcee` and the NUTS sampler of PyMC3 do not work on discrete variables.